In [1]:
import glob
import cv2
import numpy as np
import os
import shutil
import torchvision.transforms as T
import torch.nn as nn
import torch
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import random
import re

In [2]:
# シードを固定する関数
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# シードを固定
set_seed(42)

In [3]:
def make_models(model_paths):
        
    class DeepAutoencoder(nn.Module):
        def __init__(self):
            super(DeepAutoencoder, self).__init__()
            self.Encoder = nn.Sequential(
                nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(2, 2),  # 256 -> 128
                nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(2, 2),  # 128 -> 64
                nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(2, 2),  # 64 -> 32
                nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(2, 2),  # 32 -> 16
                nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(2, 2),  # 16 -> 8
            )
            self.Decoder = nn.Sequential(
                nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2),  # 8 -> 16
                nn.ReLU(),
                nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
                nn.ReLU(),
                nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2),  # 16 -> 32
                nn.ReLU(),
                nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
                nn.ReLU(),
                nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2),  # 32 -> 64
                nn.ReLU(),
                nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
                nn.ReLU(),
                nn.ConvTranspose2d(32, 16, kernel_size=2, stride=2),  # 64 -> 128
                nn.ReLU(),
                nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1),
                nn.ReLU(),
                nn.ConvTranspose2d(16, 3, kernel_size=2, stride=2),  # 128 -> 256
                nn.ReLU(),
                nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1),
            )

        def forward(self, x):
            x = self.Encoder(x)
            x = self.Decoder(x)
            return x
    
    models = []
    for model_path in model_paths:
        model = DeepAutoencoder().cuda()
        model.load_state_dict(torch.load(model_path))
        models.append(model)
    return models

In [4]:
# 画像分割サイズ
distance = 224

# 画像を分割する関数
def split(FILES):
    # 分割後の画像を分割前の画像ごとに格納
    split_images = []
    for i in range(len(FILES)):
        file = FILES[i]  # ファイル名
        img = Image.open(file)  # 画像読み込み
        img = np.array(img)  # Pillowの画像をnumpy配列に変換
        h, w = img.shape[:2]  # 画像のサイズ
        # 分割の始点
        cx = 0
        cy = 0
        for x in range(h // distance):
            for y in range(w // distance):
                # 画像の切り取り
                split_img = img[cx:cx + distance, cy:cy + distance]
                # 画像の格納
                split_images.append(Image.fromarray(split_img))  # numpy配列をPillowの画像に変換して格納
                cy += distance
            cy = 0
            cx += distance
    return split_images

In [5]:
def AE(IMGS, model_paths, threshold=30, area_threshold=1200, noise_kernel_size=5, morph_kernel_size=5, use_morphology=False, normalize=False):
    models = make_models(model_paths)
    preprocess = T.Compose([T.ToTensor()])
    model_names = [os.path.splitext(os.path.basename(path))[0] for path in model_paths]

    # result_dirを作成
    last_chars = [name.split('_')[-1][0] for name in model_names]
    result_dir = f'imgs/{"".join(last_chars)}_search_result'

    if not os.path.exists(result_dir):
        os.makedirs(result_dir)

    # 各サブフォルダを作成
    suffix = f"noise_kernel_{noise_kernel_size}"
    if use_morphology:
        suffix += f"_morph_kernel_{morph_kernel_size}"
    else:
        suffix += "_no_morph"
    if normalize:
        suffix += "_norm"
    else:
        suffix += "_nonorm"

    diff_img_dir = os.path.join(result_dir, 'new_diff_img')
    binary_img_dir = os.path.join(result_dir, f'{threshold}_new_binary_img_{suffix}')
    output_img_dir = os.path.join(result_dir, f'new_output_img')
    input_img_dir = os.path.join(result_dir, 'new_input_img')
    gray_diff_img_dir = os.path.join(result_dir, 'new_gray_diff_img')
    norm_diff_img_dir = os.path.join(result_dir, 'new_norm_diff_img')
    contour_img_dir = os.path.join(result_dir, f'{threshold}_new_contour_img_{suffix}')
    combined_img_dir = os.path.join(result_dir, f'{threshold}_combined_img_{suffix}')
    morph_img_dir = os.path.join(result_dir, 'new_morph_img')

    if not os.path.exists(diff_img_dir):
        os.makedirs(diff_img_dir)
    if not os.path.exists(contour_img_dir):
        os.makedirs(contour_img_dir)
    if not os.path.exists(binary_img_dir):
        os.makedirs(binary_img_dir)
    if not os.path.exists(output_img_dir):
        os.makedirs(output_img_dir)
    if not os.path.exists(input_img_dir):
        os.makedirs(input_img_dir)
    if not os.path.exists(combined_img_dir):
        os.makedirs(combined_img_dir)
    if not os.path.exists(gray_diff_img_dir):
        os.makedirs(gray_diff_img_dir)
    if not os.path.exists(norm_diff_img_dir):
        os.makedirs(norm_diff_img_dir)
    if not os.path.exists(morph_img_dir):
        os.makedirs(morph_img_dir)

    for img_idx, IMG in enumerate(IMGS):
        min_mse = float('inf')
        best_model_idx = -1
        best_diff = None
        best_output = None
        best_origin = None
        best_diff_mse_sum = None
        best_max_area = None
        best_binary = None
        best_contour = None
        best_morph = None

        for model_idx, model in enumerate(models):
            model.eval()
            img_tensor = preprocess(IMG).unsqueeze(0).cuda()
            with torch.no_grad():
                output = model(img_tensor)[0]
            output = output.cpu().numpy().transpose(1, 2, 0)
            output = np.uint8(np.maximum(np.minimum(output * 255, 255), 0))
            origin = np.uint8(img_tensor[0].cpu().numpy().transpose(1, 2, 0) * 255)
            diff = np.uint8(np.sqrt((output.astype(np.float32) - origin.astype(np.float32)) ** 2))
            diff_mse = np.uint8((output.astype(np.float32) - origin.astype(np.float32)) ** 2)
            diff_mse = np.sum(diff_mse)
            diff_mse_sum = diff_mse / (224 * 224 * 3)

            if diff_mse_sum < min_mse:
                min_mse = diff_mse_sum
                best_model_idx = model_idx
                best_diff = diff
                best_output = output
                best_origin = origin
                best_diff_mse_sum = diff_mse_sum

                gray_diff = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
                if normalize:
                    norm_diff = cv2.normalize(gray_diff, None, 0, 255, cv2.NORM_MINMAX)
                else:
                    norm_diff = gray_diff

                _, binary = cv2.threshold(norm_diff, threshold, 255, cv2.THRESH_BINARY)
                best_binary = binary

                noise_kernel = np.ones((noise_kernel_size, noise_kernel_size), np.uint8)
                bin_img = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, noise_kernel)

                if use_morphology:
                    morph_kernel = np.ones((morph_kernel_size, morph_kernel_size), np.uint8)
                    bin_img = cv2.dilate(bin_img, morph_kernel, iterations=2)
                    bin_img = cv2.erode(bin_img, morph_kernel, iterations=2)
                best_morph = bin_img

                contours, _ = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                contours = sorted(contours, key=cv2.contourArea, reverse=True)

                max_area = cv2.contourArea(contours[0]) if contours else 0
                best_max_area = max_area
                best_contour = contours[0] if contours else None

        # 最もMSEが小さかったモデルで以降の処理を行う
        img_counter = img_idx + 1
        img_name = f"{img_counter}.png"

        diff_path = os.path.join(diff_img_dir, img_name)
        output_path = os.path.join(output_img_dir, img_name)
        binary_path = os.path.join(binary_img_dir, img_name)
        contour_img_path = os.path.join(contour_img_dir, img_name)
        input_path = os.path.join(input_img_dir, img_name)
        gray_diff_path = os.path.join(gray_diff_img_dir, img_name)
        norm_diff_path = os.path.join(norm_diff_img_dir, img_name)
        morph_path = os.path.join(morph_img_dir, img_name)

        cv2.imwrite(diff_path, best_diff)

        output_img = cv2.cvtColor(best_output, cv2.COLOR_RGB2BGR)
        cv2.imwrite(output_path, output_img)

        input_img = cv2.cvtColor(np.array(IMG), cv2.COLOR_RGB2BGR)
        cv2.imwrite(input_path, input_img)

        cv2.imwrite(gray_diff_path, gray_diff)
        cv2.imwrite(norm_diff_path, norm_diff)
        cv2.imwrite(binary_path, best_binary)
        cv2.imwrite(morph_path, best_morph)

        # 最大面積の輪郭を塗りつぶす
        contour_img = np.zeros((best_diff.shape[0], best_diff.shape[1], 3), dtype=np.uint8)
        if best_contour is not None:
            color = [random.randint(0, 255) for _ in range(3)]
            cv2.drawContours(contour_img, [best_contour], -1, color, cv2.FILLED)
        cv2.imwrite(contour_img_path, contour_img)

        # 入力画像、膨張収縮した直後の画像、面積を算出した画像を横に並べる
        input_img = Image.open(input_path)
        morph_img = Image.open(morph_path)
        contour_img = Image.open(contour_img_path)

        combined_width = input_img.width + morph_img.width + contour_img.width
        combined_height = input_img.height
        combined_img = Image.new('RGB', (combined_width, combined_height))

        combined_img.paste(input_img, (0, 0))
        combined_img.paste(morph_img, (input_img.width, 0))
        combined_img.paste(contour_img, (input_img.width + morph_img.width, 0))

        # 物体あり/なしのラベルと最大面積、モデル名を追加
        draw = ImageDraw.Draw(combined_img)
        font = ImageFont.load_default()
        label = "YES" if best_max_area > area_threshold else "NO"
        draw.text((10, 10), label, fill=(255, 0, 0), font=font)
        draw.text((10, 30), f"Max Area: {best_max_area:.2f}", fill=(255, 0, 0), font=font)
        draw.text((10, 50), f"Model: {model_names[best_model_idx]}", fill=(255, 0, 0), font=font)

        combined_img_path = os.path.join(combined_img_dir, img_name)
        combined_img.save(combined_img_path)

    return [min_mse]

In [6]:
#一回したらコメントアウト
def move_files_to_folders(base_dir):
    # 新しいフォルダのパス
    r_model_dir = os.path.join(base_dir, 'rmodels')
    d_model_dir = os.path.join(base_dir, 'dmodels')

    # 新しいフォルダを作成
    os.makedirs(r_model_dir, exist_ok=True)
    os.makedirs(d_model_dir, exist_ok=True)

    # 元のディレクトリのパス
    fine_model_paths_dir = os.path.join(base_dir, 'fine_model_paths')

    # ディレクトリ内の全てのファイルをリストアップ
    files = os.listdir(fine_model_paths_dir)

    # ファイルを移動
    for file in files:
        file_path = os.path.join(fine_model_paths_dir, file)
        if os.path.isfile(file_path):
            if '_r' in file:
                shutil.move(file_path, os.path.join(r_model_dir, file))
            elif '_d' in file:
                shutil.move(file_path, os.path.join(d_model_dir, file))

# ベースディレクトリのパス
base_dir = 'models'

# ファイルを移動
move_files_to_folders(base_dir)

In [7]:
d_model_paths = ["models/dmodels/6048_fineAEdeepmodel_20250115_ddark.pth","models/dmodels/6048_fineAEdeepmodel_20250115_dlight.pth","models/dmodels/6048_fineAEdeepmodel_20250115_dwhite.pth"]
r_model_paths = ["models/rmodels/6048_fineAEdeepmodel_20250115_rdark.pth","models/rmodels/6048_fineAEdeepmodel_20250115_rlight.pth","models/rmodels/6048_fineAEdeepmodel_20250115_rwhite.pth"]
threshold = 80
area_threshold = 1200
files = list(glob.glob("imgs/test_img/*.JPG"))
split_images = split(files)
d_mse_lists= AE(split_images,d_model_paths,threshold,area_threshold, noise_kernel_size=3, morph_kernel_size=3, use_morphology=True, normalize=True)
r_mse_lists= AE(split_images,r_model_paths,threshold,area_threshold, noise_kernel_size=3, morph_kernel_size=3, use_morphology=True, normalize=True)

/tmp/ipykernel_1892122/309347958.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))
